# Evaluate Generative Classifiers on Non-Adversarial Samples

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))
from utils.loading import load_net
from utils.display import torch_to_np, show_imgs
from models.generative_classify import optimize_latent_cvae, gen_classify_cvae, optimize_latent, gen_classify

import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torch.nn import functional as F
import torch.optim as optim

import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.current_device())

BASE_PATH = Path('/media/rene/data/adv_gen/MNIST/mnist_normal/models')

cuda:1
0


## CVAE Results

In [7]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt10_nsamp200_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt25_nsamp200_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt50_nsamp200_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt100_nsamp200_results.pkl', "rb"))

results_d= {}
results_d[10] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results_d[25] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results_d[50] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results_d[100] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE-1_16_32_64-16-MNIST_iter50_nt100_nsamp200_deter_results.pkl', "rb"))

for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')
    
for n_t, result in results_d.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Deterministic: Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.7
Number of times: 25, Accuracy 0.765
Number of times: 50, Accuracy 0.86
Number of times: 100, Accuracy 0.86
Deterministic: Number of times: 10, Accuracy 0.775
Deterministic: Number of times: 25, Accuracy 0.885
Deterministic: Number of times: 50, Accuracy 0.855
Deterministic: Number of times: 100, Accuracy 0.91


## Paper VAE_ABS Results

In [5]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt10_nsamp200_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt25_nsamp200_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt50_nsamp200_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp200_results.pkl', "rb"))

results_d= {}
results_d[10] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results_d[25] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results_d[50] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results_d[100] = pickle.load(open(BASE_PATH/'VAE_ABS'/'VAE_ABS--8_iter50_nt100_nsamp200_deter_results.pkl', "rb"))

for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')
    
for n_t, result in results_d.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Deterministic: Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.695
Number of times: 25, Accuracy 0.835
Number of times: 50, Accuracy 0.84
Number of times: 100, Accuracy 0.89
Deterministic: Number of times: 10, Accuracy 0.84
Deterministic: Number of times: 25, Accuracy 0.855
Deterministic: Number of times: 50, Accuracy 0.895
Deterministic: Number of times: 100, Accuracy 0.94


## VAE

In [6]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt10_nsamp200_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt25_nsamp200_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt50_nsamp200_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp200_results.pkl', "rb"))
# results[200] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt200_nsamp200_results.pkl', "rb"))


results_d= {}
results_d[10] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results_d[25] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results_d[50] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results_d[100] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt100_nsamp200_deter_results.pkl', "rb"))
# results_d[200] = pickle.load(open(BASE_PATH/'VAE-1_16_32_64-16-MNIST'/'VAE-1_16_32_64-16_iter50_nt200_nsamp200_deter_results.pkl', "rb"))


for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')
    
for n_t, result in results_d.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Deterministic: Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.755
Number of times: 25, Accuracy 0.765
Number of times: 50, Accuracy 0.86
Number of times: 100, Accuracy 0.875
Deterministic: Number of times: 10, Accuracy 0.805
Deterministic: Number of times: 25, Accuracy 0.85
Deterministic: Number of times: 50, Accuracy 0.9
Deterministic: Number of times: 100, Accuracy 0.91


## CVAE ABS

In [8]:
results= {}
results[10] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE_ABS-8_iter50_nt10_nsamp200_deter_results.pkl', "rb"))
results[25] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE_ABS-8_iter50_nt25_nsamp200_deter_results.pkl', "rb"))
results[50] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE_ABS-8_iter50_nt50_nsamp200_deter_results.pkl', "rb"))
results[100] = pickle.load(open(BASE_PATH/'CVAE'/'CVAE_ABS-8_iter50_nt100_nsamp200_deter_results.pkl', "rb"))

for n_t, result in results.items():
    acc = len(result[result['predicted_label']==result['true_label']])/len(result)
    print(f'Number of times: {n_t}, Accuracy {acc}')

Number of times: 10, Accuracy 0.76
Number of times: 25, Accuracy 0.83
Number of times: 50, Accuracy 0.86
Number of times: 100, Accuracy 0.855
